# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Setup Mediapipe

In [2]:
mp_hands = mp.solutions.hands # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    if results:
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

In [5]:
WIDTH = 854
HEIGHT = 480
def rescale_frame(frame):
    dim = (WIDTH, HEIGHT)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

# 3. Setup Keypoint Processing

In [6]:
def extract_keypoints(results):
    lh = np.zeros(21*3)
    if (results.multi_hand_landmarks):
        if (results.multi_hand_landmarks[0]):
            # print([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark])
            min_x = 9999
            min_y = 9999
            min_z = 9999
            for res in results.multi_hand_landmarks[0].landmark :
                # print(res)
                if (res.x < min_x):
                    min_x = res.x
                if (res.y < min_y):
                    min_y = res.y
                if (res.z < min_z):
                    min_z = res.z
            # print (min_x)
            # print (min_y)
            # print (min_z)
            for res in results.multi_hand_landmarks[0].landmark :
                res.x -= min_x
                res.y -= min_y
                res.z -= min_z
                                           
            lh = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark]).flatten()
            # print(lh)
    rh = np.zeros(21*3)
    if (results.multi_hand_landmarks):
        if (len(results.multi_hand_landmarks) > 1):
            min_x = 9999
            min_y = 9999
            min_z = 9999
            for res in results.multi_hand_landmarks[1].landmark :
                if (res.x < min_x):
                    min_x = res.x
                if (res.y < min_y):
                    min_y = res.y
                if (res.z < min_z):
                    min_z = res.z
            for res in results.multi_hand_landmarks[1].landmark :
                res.x -= min_x
                res.y -= min_y
                res.z -= min_z
            rh = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[1].landmark]).flatten()
    return np.concatenate([lh, rh])

# 4. Define Training Folders

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
TRAINING_PATH = os.path.join('MS-ASL/MS-ASL/videos')

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])
actions = np.array(['eat', 'fish', 'nice', 'milk', 'teacher', 'finish', 'cousin', 'orange', 'yes', 'student', 'sister', 'friend', 'yellow',
                   'white', 'what', 'water', 'want', 'tired', 'pencil', 'mother', 'like', 'drink', 'again', 'table', 'school', 'no', 'help', 'blue', 'spring',
                   'doctor', 'deaf', 'red', 'father', 'black'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

# 5. Import Train Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
sequences, labels = [], []
for action in actions:
    # print(action)
    for sequence in range(start_folder, no_sequences + 1):
        window = []
        # print(sequence)
        for frame_num in range(1, sequence_length + 1):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            if not (res.any()):
                print(action);
                print(sequence)
            window.append(res)
            # print(frame_num)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 6. Build LSTM Neural Network

In [12]:
import tensorflow as tf

tf.config.threading.set_intra_op_parallelism_threads(6)
tf.config.threading.set_inter_op_parallelism_threads(2)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(np.array(sequences[0]).shape)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# 7. Load Trained Model Weights

In [16]:
model.load_weights('msasl1.h5')

# 8. Setup Visualization

In [17]:
from scipy import stats

In [18]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,117,245), (16,117,245)] * 10
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        
    return output_frame

In [19]:
# 1. New detection variables
predictions = []
res = []
sequence = []
threshold = 0.85

import time

predict = True

def predict_word() :
    global res
    while predict :
        time.sleep(0.5)
        if (len(sequence) == sequence_length) :
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=None)[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        else:
            res = []

In [20]:
frame = np.zeros(1)
run_cv_thread = True
def run_cv():
    global frame
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    
    while run_cv_thread:
        ret, frame_new = cap.read()
        if (ret == False):
            break
        frame = frame_new
    cap.release()

# 9. Run Model

In [21]:
import threading

t1 = threading.Thread(target=predict_word, args=())
t1.start()
t2 = threading.Thread(target=run_cv, args=())
t2.start()

In [ ]:
frames_wo_points = 0

with mp_hands.Hands(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    predictions = []
    res = []
    sequence = []
    sentence = []
    while (frame.any()):

        frame = rescale_frame(frame)

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        # print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        if np.any(keypoints):
            frames_wo_points = 0
            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]
        else:
            frames_wo_points += 1
            if (frames_wo_points >= 10):
                sequence = []

        #3. Viz logic
        # print(np.any(predictions) & np.any(res))
        if (np.any(predictions) & np.any(res)) :
            if np.unique(predictions[-3:])[0]==np.argmax(res): 
                # print(res[np.argmax(res)])
                # print(np.argmax(res))
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                    # print(sentence)

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        if (np.any(predictions)):
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screenP
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
       
    # cap.release()
    cv2.destroyAllWindows()

# 10. Stop Processes

In [29]:
predict = False
t1.join()
predict = True
run_cv_thread = False
t2.join()
run_cv_thread = True
cv2.destroyAllWindows()

NameError: name 'cap' is not defined